In [4]:
import os
from os import environ
import numpy as np

import pandas as pd
pd.set_option('display.max_colwidth', 0)

# https://pypi.org/project/python-dotenv/
from dotenv import load_dotenv, find_dotenv
dot_env_file_exist = load_dotenv(find_dotenv()) # read local .env file
# Returns true or false if .env exists in current directory
print('dot_env_file_exist: ', dot_env_file_exist)

import openai
import json

dot_env_file_exist:  True


In [6]:
# PYTHON Solution: Read the key directly from the .env file
import openai
def python_get_dotenv_data(parms):
    with open('.env', 'r') as reader:
        out = reader.readlines()
    for i in parms:
        for j in range(len(out)):
            ind = out[j].rfind(i)
            if ind != -1:
                st = ind+(2+len(i))
                end = len(out[j]) - 2
                globals()[f'{i}'] = out[j][st:end]

# Save key as a Python variable
parms = ['OPENAI_API_KEY']
python_get_dotenv_data(parms)
openai.api_key = OPENAI_API_KEY
print(openai.api_key)

sk-AIpV99p7zCf0nQ0ebo0tT3BlbkFJllwV2ZBKHiiXkeenSGp


In [7]:
# Define the client
from openai import OpenAI
import openai
client = openai.api_key

In [15]:
# Starten des Upload-Prozess Training-Daten
import openai
client = OpenAI()

my_training_data = client.files.create(
    file=open("data/training_test_set.jsonl", "rb"),
    purpose='fine-tune'
)

In [17]:
print(my_training_data.id)

file-yJYIO2sHWmrGWGYhKoJTUGI9


In [19]:
# Starten des Upload-Prozess Validierung-Daten
import openai
client = OpenAI()

my_eval_data = client.files.create(
    file=open("data/validation_test_set.jsonl", "rb"),
    purpose='fine-tune'
)

In [20]:
print(my_eval_data.id)

file-FEfZVH9BPHodBaQLXQsYTo0Z


In [21]:
# Start das Finetuning
from openai import OpenAI
client = OpenAI()

response= client.fine_tuning.jobs.create(
  training_file=my_training_data.id,
  validation_file=my_eval_data.id,
  model="gpt-3.5-turbo"
)

response

FineTuningJob(id='ftjob-ZdaAd5mj3iHHbZ9FXiUVoNo4', created_at=1711979551, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-4iR8mcaw4LEVCNvoGGcPHEEv', result_files=[], status='validating_files', trained_tokens=None, training_file='file-yJYIO2sHWmrGWGYhKoJTUGI9', validation_file='file-FEfZVH9BPHodBaQLXQsYTo0Z', user_provided_suffix=None)

In [22]:
job_id = response.id
job_id

'ftjob-ZdaAd5mj3iHHbZ9FXiUVoNo4'

In [26]:
# Zeige den Status des Fine-Tuning an
the_job = client.fine_tuning.jobs.retrieve(job_id)
the_job

FineTuningJob(id='ftjob-ZdaAd5mj3iHHbZ9FXiUVoNo4', created_at=1711979551, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:christian-kolb::99CSQB6v', finished_at=1711979865, hyperparameters=Hyperparameters(n_epochs=10, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-4iR8mcaw4LEVCNvoGGcPHEEv', result_files=['file-dUFLE9R4x9iY9PdGSrLOW1TI'], status='succeeded', trained_tokens=5470, training_file='file-yJYIO2sHWmrGWGYhKoJTUGI9', validation_file='file-FEfZVH9BPHodBaQLXQsYTo0Z', user_provided_suffix=None)

In [27]:
# Zeig die Job ID
print("Job ID: ", the_job.id)
#Zeig den Status des Jobs
print("Job Status: ", the_job.status)
# Zeig die Anzahl der trainierten Tokens an
print("Die trainierten Tokens: ", the_job.trained_tokens)
# Zeige den Namen von dem neuen Fintuned Modell an
print("Name des neuen Fintuned Modells: ", the_job.fine_tuned_model)

Job ID:  ftjob-ZdaAd5mj3iHHbZ9FXiUVoNo4
Job Status:  succeeded
Die trainierten Tokens:  5470
Name des neuen Fintuned Modells:  ft:gpt-3.5-turbo-0125:christian-kolb::99CSQB6v


Nützliche Codes

In [ ]:
#Stoppe einen Job
client.fine_tuning.jobs.cancel ("ftjob-abc1223")

#Lösche eine finetuned Modell
client.models.delete ("ftmodel-abc1223")

Teste das neue Model

In [ ]:
def test_new_model(prompt):
    #Stelle einer Antwort von dem Modell
    response = client.chat.completions.create(
        model="ft:gpt-3.5-turbo-0125:christian-kolb::99CSQB6v",
        messages=[
            {
                "role": "system", "content": "Clippy, the witty chatbot, blending facts with a touch of sarcasm."
                "role": "user", "content": prompt
                }
        ]
    )

In [ ]:
#Cancel a Job
client.fine_tuning.jobs.cancel("job_123")

#Lösche a Fine.tuned Model
client.models.delete("ft:model_name")